In [11]:
import pandas as pd
import pickle as pk
from tensorflow.keras.models import load_model

In [12]:
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pk.load(file)

with open('oneHot_encoder_geo.pkl', 'rb') as file:
    oneHot_encoder_geo = pk.load(file)

with open('scalar.pkl', 'rb') as file:
    scalar = pk.load(file)

model = load_model('model.h5')

2026-01-21 14:35:32.793044: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2026-01-21 14:35:32.793120: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2026-01-21 14:35:32.793130: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.72 GB
I0000 00:00:1768995332.793149 5087809 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1768995332.793183 5087809 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])

In [7]:
encoder_geo = oneHot_encoder_geo.transform(input_data[['Geography']]).toarray()

In [8]:
input_data = pd.concat([input_data.drop('Geography', axis=1), 
                        pd.DataFrame(encoder_geo, columns=oneHot_encoder_geo.get_feature_names_out())], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
## Scaling the input data
input_scaled=scalar.transform(input_data)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


array([[0.06780807]], dtype=float32)

In [19]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.06780807)

In [20]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.


In [21]:
!pip install streamlit